# Detection of TOXicity in comments in Spanish (DETOXIS 2021)

## SESIÓN 2.2: Clasificación

### Realizado por Álvaro Mazcuñán y Miquel Marín

#### Librerías

Se importan las mismas librerías que se utilizaron en la parte anterior

In [43]:
import pandas as pd
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

from gensim.models import Word2Vec

#### Datos de DETOXIS

A continuación se cargan los datos tal y como se hizo en la anterior entrega

In [2]:
data = pd.read_csv('C:/Users/mique/Documents/CURSO 3 - SEMESTRE B/LENGUAJE NATURAL Y RECUPERACION DE LA INFORMACION/PRACTICAS/DATASET_DETOXIS.csv')
data

,topic,thread_id,comment_id,reply_to,comment_level,comment,argumentation,constructiveness,positive_stance,negative_stance,...,target_group,stereotype,sarcasm,mockery,insult,improper_language,aggressiveness,intolerance,toxicity,toxicity_level
0,CR,0_000,0_002,0_002,1,Pensó: Zumo para restar.,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
1,CR,0_001,0_003,0_003,1,Como les gusta el afeitado en seco a esta gente.,0,0,0,0,...,1,1,1,1,0,0,0,0,1,1
2,CR,0_002,0_004,0_004,1,"asi me gusta, que se maten entre ellos y en al...",0,0,0,0,...,1,0,0,0,0,0,1,1,1,2
3,CR,0_003,0_005,0_005,1,"Loss mas valientes, los que mejor cortan nuest...",0,0,0,0,...,1,0,1,1,0,0,0,0,1,1
4,CR,0_004,0_006,0_006,1,Costumbres...,0,0,0,0,...,1,1,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458,MI,20_134,20_164,20_164,1,Ya decía yo que veía menos moros,0,0,0,0,...,1,0,0,1,1,0,0,0,1,1
3459,MI,20_006,20_165,20_008,2,+1. Como lo sabes...,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3460,MI,20_135,20_166,20_166,1,"Seguirán cobrando paguitas en Marruecos,expoli...",0,0,0,0,...,1,1,0,0,0,0,0,1,1,1
3461,MI,20_136,20_167,20_167,1,"pobres, se arriesgan en pateras porque huyen d...",0,0,0,0,...,1,0,0,1,0,0,0,0,1,1


#### Subset de variables para el análisis

In [3]:
sample_data = data[["comment", "toxicity","toxicity_level"]]
sample_data

,comment,toxicity,toxicity_level
0,Pensó: Zumo para restar.,1,1
1,Como les gusta el afeitado en seco a esta gente.,1,1
2,"asi me gusta, que se maten entre ellos y en al...",1,2
3,"Loss mas valientes, los que mejor cortan nuest...",1,1
4,Costumbres...,1,1
...,...,...,...
3458,Ya decía yo que veía menos moros,1,1
3459,+1. Como lo sabes...,0,0
3460,"Seguirán cobrando paguitas en Marruecos,expoli...",1,1
3461,"pobres, se arriesgan en pateras porque huyen d...",1,1


#### Leer tweets y preprocesado 

In [4]:
def tweet_preprocessing_not_tokenized(tweet):
    tweet = tweet.lower() # Se empieza pasando todos los mensajes a minúsculas
    tweet = re.sub(r"http\S+|www\S+|https\S+", "" ,tweet , flags=re.MULTILINE) # Quitar URLs
    tweet = re.sub(r"\@\w+|\#", "", tweet) # Quitar @ y #
    tweet = re.sub(r"[\U00010000-\U0010ffff]|:\)|:\(|XD|xD|;\)|:,\(|:D|D:", "", tweet) # Quitar emojis y emoticones
    tweet = tweet.translate(str.maketrans('', '', string.punctuation)) # Quitar signos de puntuación
    tokenized_tweets = word_tokenize(tweet)
    filtered_tweets = [word for word in tokenized_tweets if not word in set(stopwords.words('spanish'))] # Quitar stopwords y filtrar
    
    stemming = PorterStemmer() # Inicializamos PorterStemmer para obtener la raíz de cada una de las palabras
    stemmed_tweets = [stemming.stem(word) for word in filtered_tweets]
    lemmatization = WordNetLemmatizer() # Inicializamos el Lemmatizer para obtener los lemas de las palabras
    lemma_tweets = [lemmatization.lemmatize(word, pos='a') for word in stemmed_tweets] 
    return " ".join(lemma_tweets) # NO TOKENIZADO

preprocessing = lambda x: tweet_preprocessing_not_tokenized(x)

In [5]:
sample_data['comment'] = pd.DataFrame(sample_data["comment"].apply(preprocessing))

<ipython-input-5-3c4e5d0f187f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_data['comment'] = pd.DataFrame(sample_data["comment"].apply(preprocessing))


In [7]:
sample_data["comment"][0]

'pensó zumo restar'

#### Problema del desbalance de clases

Debido a que se van a usar algoritmos de clasificación se tendría que estudiar, para los dos variables de `toxicity` y `toxicity_level`, si dichas clases están balanceadas o no debido a que en problemas de clasificación se suelen encontrar que en el conjunto de datos de entrenamiento una de las clases es minoritaria, es decir, en ella hay muy pocas muestras. Esto puede llegar a afectar a los algoritmos en su proceso de generalización y, en consecuencia, no poder diferenciar una clase de la otra.

In [8]:
sample_data["toxicity"].value_counts()

0    2316
1    1147
Name: toxicity, dtype: int64

In [9]:
sample_data["toxicity_level"].value_counts()

0    2317
1     808
2     269
3      69
Name: toxicity_level, dtype: int64

Tal y como se puede observar, en la variable `toxicity`, existen 2316 instancias de la clase 0. Por otra parte, la clase 1 contiene 1147. En este caso, aunque la clase 0 tenga el doble de observaciones, consideramos que no existe un problema de desbalanceo de clases ya que existen varias instancias de cada una de las dos clases. Sin embargo, en la variable `toxicity_level` sí que existe un desbalanceo de clases debido a que en las clases 2 y 3 existen pocas observaciones respecto a las otras dos. 
Por lo tanto, se ha decidido, una vez se evaluen los modelos correspondientes, balancear dicha variable.

#### Dividir el corpus en conjunto de entrenamiento y test

Hay que tener claro que, en primer lugar, se van a realizar los modelos de extracción de características y de Machine Learning para la variable `toxicity`.

In [10]:
train_X, test_X, train_Y, test_Y = train_test_split(sample_data['comment'], sample_data['toxicity'], test_size=0.3)

### Extracción de características

#### Term Frequency - Inverse Document Frequency (TF-IDF)

Una vez se ha dividido el corpus en train y test, se pasa a realizar un TF-IDF para luego entrenar los datos con una serie de modelos que se comentarán posteriormente.

In [13]:
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(sample_data['comment'])
train_X_Tfidf = tfidf_vect.transform(train_X)
test_X_Tfidf = tfidf_vect.transform(test_X)

In [18]:
print(tfidf_vect.vocabulary_)

 'demonizar': 3584, 'procesión': 9644, 'cabalgata': 1751, 'festival': 5341, 'camion': 1872, 'clausura': 2302, 'liberada': 7105, 'bofetada': 1617, 'hocico': 6030, 'arabiacatar': 1048, 'amado': 743, 'flujo': 5424, 'financiación': 5379, 'conducto': 2653, 'empleamo': 4426, 'paraíso': 8771, 'alegoría': 636, 'nazismo': 8198, 'implementarlo': 6247, 'inductor': 6420, 'prevenir': 9591, 'mordaza': 7984, 'secuestrar': 10911, 'sagrado': 10780, 'contradicen': 2843, 'sonamo': 11263, 'parafernalia': 8757, 'teatro': 11569, 'deberiamo': 3368, 'gustaria': 5824, 'daesh': 3314, 'pernicisoso': 9081, 'nutren': 8374, 'siembran': 11078, 'dividiendo': 4108, 'enfrentando': 4530, 'sermon': 11011, 'designa': 3778, 'radicalismo': 10073, 'tolerando': 11767, 'lavar': 7016, 'independientment': 6384, 'esperaban': 4845, 'escapan': 4725, 'creyendo': 3064, 'vision': 12432, 'neutra': 8267, 'traeria': 11867, 'cientifismo': 2238, 'carent': 1954, 'quiza': 10050, 'satisfaccion': 10884, 'mejormejor': 7688, 'ajustas': 596, 'ine

### Evaluación de modelos - Variable `toxicity`

#### Support Vector Machines (SVM)

Una vez obtenidos los coeficientes de importancia de cada uno de los términos, se pasa a realizar el entrenamiento de un modelo, en este caso se va a empezar por el modelo de Máquinas de Soporte Vectorial

In [34]:
svm_clf = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svm_clf.fit(train_X_Tfidf,train_Y)
predictions_SVM = svm_clf.predict(test_X_Tfidf)

In [35]:
score_svm = f1_score(test_Y, predictions_SVM, average='macro')
score_svm

0.6812138588705624

Una vez entrenado el corpus con TF-IDF junto con el algoritmo de SVM, se puede observar que su F1-Score es de 0.68. Para una primera evaluación de dicho modelo, se puede decir que su resultado es mejorable pero no es un valor del todo malo ya que el valor óptimo de dicha métrica sería de 1.

#### Decision Tree

In [39]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(train_X_Tfidf,train_Y)
predictions_tree = tree_clf.predict(test_X_Tfidf)

In [40]:
score_tree = f1_score(test_Y, predictions_tree, average='macro')
score_tree

0.6719990207195135

En este caso se ha entrenado un árbol de decisión (también con la previa extracción de características de TF-IDF) y se puede observar como su F1-Score es muy similar al del modelo de Máquinas de Soporte Vectorial.

#### Logistic Regression

In [41]:
logreg_clf = LogisticRegression()
logreg_clf.fit(train_X_Tfidf,train_Y)
predictions_logreg = logreg_clf.predict(test_X_Tfidf)

In [42]:
score_logreg = f1_score(test_Y, predictions_logreg, average='macro')
score_logreg

0.5718558432742894

En tercer caso, se ha evaluado un modelo de regresión logística y su valor de F1-Score es muy inferior al de los anteriores modelos realizados (0.57 aproximadamente)

#### Perceptrón multicapa

In [44]:
mlp_clf = MLPClassifier(random_state=1, max_iter=300)
mlp_clf.fit(train_X_Tfidf,train_Y)
predictions_mlp = mlp_clf.predict(test_X_Tfidf)

No hace falta ejecutarlo

In [45]:
score_mlp = f1_score(test_Y, predictions_mlp, average='macro')
score_mlp

0.670053244909753

Finalmente, se ha evaluado un perceptrón multicapa y se ha obtenido un F1-Score muy similar a los modelos de Regresión Logística y Máquinas de Soporte Vectorial

Todo lo que se ha realizado anteriormente ha sido mediante la variable `toxicity`. Debido a que hay dos tareas, a continuación, se pasa a realizar el mismo proceso para la variable desglosada de `toxicity_level`.

In [46]:
train_X_, test_X_, train_Y_, test_Y_ = train_test_split(sample_data['comment'], sample_data['toxicity_level'], test_size=0.3)

In [47]:
tfidf_vect_ = TfidfVectorizer()
tfidf_vect_.fit(sample_data['comment'])
train_X_Tfidf_ = tfidf_vect_.transform(train_X_)
test_X_Tfidf_ = tfidf_vect_.transform(test_X_)

### Evaluación de modelos - Variable `toxicity_level`

#### Support Vector Machines (SVM)

In [61]:
svm_clf_ = SVC(C=1.0, kernel='linear', degree=3, gamma='auto', class_weight="balanced")
svm_clf_.fit(train_X_Tfidf_,train_Y_)
predictions_SVM_ = svm_clf.predict(test_X_Tfidf_)

In [62]:
score_svm_ = f1_score(test_Y_, predictions_SVM_, average='macro')
score_svm_

0.39164858927476953

En este caso, utilizando la variable `toxicity_level`, se puede observar como el F1-Score ha disminuido radicalmente respecto a la anterior variable (0.40 aproximadamente)

#### Decision Tree

In [63]:
tree_clf_ = DecisionTreeClassifier(class_weight="balanced")
tree_clf_.fit(train_X_Tfidf_,train_Y_)
predictions_tree_ = tree_clf_.predict(test_X_Tfidf_)

In [64]:
score_tree_ = f1_score(test_Y_, predictions_tree_, average='macro')
score_tree_

0.31845286557190056

El F1-Score utilizando un árbol de decisión es de 0.31, un resultado bastante pobre.

#### Logistic Regression

In [66]:
logreg_clf_ = LogisticRegression(class_weight="balanced")
logreg_clf_.fit(train_X_Tfidf_,train_Y_)
predictions_logreg_ = logreg_clf_.predict(test_X_Tfidf_)

In [67]:
score_logreg_ = f1_score(test_Y_, predictions_logreg_, average='macro')
score_logreg_

0.39045786513335085

#### Perceptrón multicapa

In [70]:
mlp_clf_ = MLPClassifier(random_state=1, max_iter=300)
mlp_clf_.fit(train_X_Tfidf_,train_Y_)
predictions_mlp_ = mlp_clf_.predict(test_X_Tfidf_)

In [71]:
score_mlp_ = f1_score(test_Y_, predictions_mlp_, average='macro')
score_mlp_

0.288129879836838